##Validation data를 포함하여 Fine-tuning 해보기

In [ ]:
!pip install datasets

In [ ]:
!pip install evaluate

In [6]:
import os
import sys
import math
import torch
import wandb
import logging
import datasets
import argparse
import evaluate
import transformers

from typing import Optional
from itertools import chain
from dataclasses import dataclass, field

In [9]:
from datasets import load_dataset
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    default_data_collator
)
from transformers.trainer_utils import get_last_checkpoint


In [10]:
#device setting
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.backends.cuda.is_built():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

cuda


In [12]:
wandb.init(project='Hanghae99')
wandb.run.name = 'gpt-finetuning'

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kimguangdong (kimguangdong-js) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [13]:
@dataclass
class Arguments:
    model_name_or_path: Optional[str] = field(default=None)
    torch_dtype: Optional[str] = field(default=None, metadata={'choices': ['auto', 'bfloat16', 'float16', 'float32']})

    dataset_name: Optional[str] = field(default=None)
    dataset_config_name: Optional[str] = field(default=None)
    block_size: int = field(default=1024)
    num_workers: Optional[int] = field(default=None)

필드 설명:
model_name_or_path:
사용할 사전 학습 모델의 경로나 이름 (e.g., "bert-base-uncased", "facebook/opt-1.3b").None이면 기본값을 사용하거나 별도로 설정해야 함.

torch_dtype:
모델을 어떤 데이터 타입으로 로드할지 결정 (auto, bfloat16, float16, float32 중 선택 가능). bfloat16이나 float16을 선택하면 VRAM 사용량이 줄어듦.

dataset_name:
Fine-tuning에 사용할 Hugging Face 데이터셋의 이름 (e.g., "wikitext", "glue").

dataset_config_name: 데이터셋의 특정 설정 (예: "wikitext-2-v1").
block_size: 입력 시퀀스 길이 (기본값: 1024).
모델에 따라 적절한 길이 조정 필요 (예: BERT의 경우 512 제한).

num_workers:
데이터 로딩 및 전처리 시 사용할 worker 수.
None이면 기본값(os.cpu_count())이 사용됨.

In [14]:
args = Arguments(model_name_or_path="distilbert/distilbert-base-uncased",
                 torch_dtype='auto',
                 dataset_name="nyu-mll/glue",
                 dataset_config_name="mnli",
                 num_workers=1
                 )

In [17]:
lr = 2e-5
n_epochs = 2
batch_size = 1
batch_size = 32

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',                 # 모델, log 등을 저장할 directory
    num_train_epochs= n_epochs,             # epoch 수
    per_device_train_batch_size=batch_size, # training data의 batch size
    per_device_eval_batch_size=batch_size,  # validation data의 batch size
    logging_strategy="epoch",               # Epoch가 끝날 때마다 training loss 등을 log하라는 의미
    do_train=True,                          # 학습을 진행하겠다는 의미
    do_eval=True,                           # 학습 중간에 validation data에 대한 평가를 수행하겠다는 의미
    eval_strategy="epoch",                  # 매 epoch가 끝날 때마다 validation data에 대한 평가를 수행한다는 의미
    save_strategy="epoch",                  # 매 epoch가 끝날 때마다 모델을 저장하겠다는 의미
    learning_rate=lr,                       # optimizer에 사용할 learning rate
    load_best_model_at_end=True,            # 학습이 끝난 후, validation data에 대한 성능이 가장 좋은 모델을 채택하겠다는 의미
    report_to="none"
)



# 사용자 정의  DeBERTa 매개변수
config = DebertaConfig(
    hidden_size=64,             # layer의 기본 hidden dimension
    intermediate_size=64,       # FFN layer의 중간 hidden dimension
    num_attention_heads=4,      # Multi-head attention에서 사용하는 head 개수
    num_hidden_layers=2,        # 은닉층 수량
    num_labels=3                # 마지막에 예측해야 하는 분류 문제의 class 개수
)



In [ ]:
parser = HfArgumentParser((Arguments, TrainingArguments))
args, training_args = parser.parse_args_into_dataclasses()

##데이터 업로드
Hugging Face Hub 에서 데이터 가져오기

In [19]:
raw_datasets = load_dataset(
    args.dataset_name,
    args.dataset_config_name
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/52.2M [00:00<?, ?B/s]

(…)alidation_matched-00000-of-00001.parquet:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

(…)dation_mismatched-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

## Tokenizer

In [23]:
from transformers import DebertaConfig, AutoModelForSequenceClassification


config = AutoConfig.from_pretrained(args.model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path)
model = model = AutoModelForSequenceClassification.from_config(config)